In [0]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing.text import Tokenizer
import spacy
import numpy as np
import sys
import io

import warnings
warnings.filterwarnings('ignore')

In [3]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.3MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=4763e40d1af396c8ce9e5ba6c7efac83ed3a61b7479721d17993d337f07aba4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7ew8e8b/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import en_core_web_md
nlp = en_core_web_md.load()


In [0]:
nlp.max_length = 16582681

In [7]:
# glove upload
from google.colab import files
uploaded = files.upload()

Saving glove.6B.50d.txt to glove.6B.50d.txt


In [8]:
# text file upload
from google.colab import files
uploaded = files.upload()

Saving text.txt to text.txt


In [9]:
embeddings = {}
f = open('glove.6B.50d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

print(f'Found {len(embeddings)} word vectors.')

Found 400000 word vectors.


In [10]:
embeddings['president'] 

array([-0.11875  ,  0.6722   ,  0.19444  ,  0.55269  ,  0.53698  ,
       -0.37237  , -0.73494  , -0.30575  , -0.92601  , -0.43276  ,
        0.026956 ,  0.66861  , -0.79097  , -0.015932 ,  0.53918  ,
        0.30341  , -0.67042  ,  0.0051129,  0.62272  , -0.55823  ,
       -0.10887  ,  0.57305  , -0.016149 , -1.1889   , -0.24318  ,
       -2.6289   ,  0.41262  , -0.12904  , -1.3238   ,  0.64731  ,
        2.3595   ,  0.34048  , -1.9889   , -0.79084  , -0.79739  ,
       -0.87998  , -0.72991  ,  0.011697 ,  0.090612 , -0.17287  ,
       -0.83274  ,  1.1932   , -0.75211  , -1.1603   , -0.10074  ,
        0.60224  , -1.3739   ,  0.33674  , -0.31224  ,  0.097583 ],
      dtype=float32)

In [0]:
with open('text.txt', 'r') as f:
  text = f.read()

In [0]:
nlp.add_pipe(nlp.create_pipe('sentencizer'))
doc = nlp(text, disable=['ner', 'parser'])

In [0]:
sentences = [sent.string.strip() for sent in doc.sents]

In [14]:
sentences[0:20]

['Transcribed from the 1915 Martin Secker edition by David Price, email ccx074@pglaf.org                            [Picture: Book cover]                                    THE LESSON OF                                 THE MASTER                                 BY HENRY JAMES                        [Picture: Decorative graphic]                                  * * * * *                            LONDON: MARTIN SECKER                      NUMBER FIVE JOHN STREET ADELPHI                                  * * * * *                       This edition first printed 1915                                  * * * * *     I   HE had been told the ladies were at church, but this was corrected by what he saw from the top of the steps—they descended from a great height in two arms, with a circular sweep of the most charming effect—at the threshold of the door which, from the long bright gallery, overlooked the immense lawn.',
 'Three gentlemen, on the grass, at a distance, sat under the great trees,

In [0]:
maxlen = 20
step = 1
max_num_word = 10000

samples_sentences = []
next_word = []

In [0]:
# tokenze and transform each sentence into a sequence of ints
tokenizer = Tokenizer(num_words=max_num_word)
tokenizer.fit_on_texts(sentences)
list_tokenized_train = tokenizer.texts_to_sequences(sentences)

In [17]:
len(tokenizer.word_index)

45332

In [18]:
len(sentences)

150009

In [19]:
len(list_tokenized_train)

150009

In [0]:
token_word = []
for line in range(0, len(sentences)):
  that_sentences = list_tokenized_train[line]
  for i in range(0, len(that_sentences)):
    token_word.append(that_sentences[i])

In [32]:
len(token_word)

2890983

In [23]:
for i in range(0, len(token_word) - maxlen, step):
  samples_sentences.append(token_word[i:i + maxlen])
  next_word.append(token_word[i + maxlen])
print('Number of sentences:', len(samples_sentences))

Number of sentences: 2890963


In [0]:
# normalize sequences
x = np.asarray(samples_sentences).astype('float32')/max_num_word
